In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

NameError: name 'ids' is not defined

In [3]:
results

{}

In [4]:
print("Hello World :)")

Hello World :)


In [5]:
get_ipython().system('pip install db-dtypes')

In [6]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import matplotlib.pyplot as plt
import numpy as np

In [7]:
# Constants
PRED_WEEK = 40

In [8]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [9]:
query_get_tables = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""
query_job = bigquery_client.query(query_get_tables)
query_job.to_dataframe().head()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,ironhacks-data,ironhacks_competition,wage_data,BASE TABLE,YES,NO,2022-11-29 17:00:10.578000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
1,ironhacks-data,ironhacks_competition,prediction_list,BASE TABLE,YES,NO,2022-11-29 17:00:10.722000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
2,ironhacks-data,ironhacks_competition,unemployment_data,BASE TABLE,YES,NO,2022-11-29 17:00:14.588000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT


In [10]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [11]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0


In [12]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [13]:
query_job = bigquery_client.query(query)
prediction_list_data = query_job.to_dataframe()
prediction_list_data.head(2)

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44


In [14]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [15]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head(2)

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.00
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.25


In [16]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
WHERE uu_id IN (SELECT uu_id
                FROM `ironhacks-data.ironhacks_competition.prediction_list`
                WHERE week_number = (SELECT MAX(week_number) FROM `ironhacks-data.ironhacks_competition.prediction_list`))
ORDER BY uu_id,week_number
"""

In [17]:
query_job = bigquery_client.query(query)
relevant_unemployment_df = query_job.to_dataframe()
relevant_unemployment_df.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [18]:
relevant_unemployment_df.head(3)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10
2,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [19]:
# Drop duplicate columns
relevant_unemployment_df.drop_duplicates(inplace=True)
relevant_unemployment_df.drop(['timeperiod', 'tract_name'], axis=1, inplace=True)

In [20]:
# Drop columns with excessive null values
# NOTE: Revisit this, these columns may still be useful, especially those that aren't missing too many values
relevant_unemployment_df.dropna(axis='columns', inplace=True)
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()
relevant_unemployment_df['countyfips'] = relevant_unemployment_df['countyfips'].astype(str)
relevant_unemployment_df['tract'] = relevant_unemployment_df['tract'].astype(str)
relevant_unemployment_df['real_data'] = True
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()

Add rows such that every uu_id has rows for weeks 1:max(week_number)

In [21]:
# Add Wage data
relevant_unemployment_df['wage_data'] = relevant_unemployment_df['uu_id'].map(wage_data.set_index('uu_id')['average_wage'])
relevant_unemployment_df.info()
bad_wage_uu_ids = list(set(relevant_unemployment_df.loc[relevant_unemployment_df['wage_data'].isna(),'uu_id']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13977 entries, 0 to 16831
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   13977 non-null  string 
 1   week_number             13977 non-null  Int64  
 2   countyfips              13977 non-null  string 
 3   tract                   13977 non-null  string 
 4   total_claims            13977 non-null  Int64  
 5   top_category_employer1  13977 non-null  string 
 6   top_category_employer2  13977 non-null  string 
 7   top_category_employer3  13977 non-null  string 
 8   real_data               13977 non-null  boolean
 9   wage_data               13977 non-null  float64
dtypes: Int64(2), boolean(1), float64(1), string(6)
memory usage: 1.1 MB


In [22]:
for uuid in set(relevant_unemployment_df["uu_id"]):
    num_tracts = 0
    tracts=[]
    for tract in set(relevant_unemployment_df.loc[relevant_unemployment_df["uu_id"]==uuid,'wage_data']):
        num_tracts += 1
        tracts.append(tract)
    if num_tracts > 1:
        print(uuid,num_tracts,tracts)
relevant_unemployment_df

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,001cd9ae23064d7f0fd3cd327c873d8d,31,18099,20102,34,31-33,23,81,True,10484.333333
1,001cd9ae23064d7f0fd3cd327c873d8d,33,18099,20102,10,31-33,23,81,True,10484.333333
3,001cd9ae23064d7f0fd3cd327c873d8d,34,18099,20102,25,62,31-33,23,True,10484.333333
5,001cd9ae23064d7f0fd3cd327c873d8d,35,18099,20102,10,31-33,61,23,True,10484.333333
7,001cd9ae23064d7f0fd3cd327c873d8d,36,18099,20102,10,31-33,23,81,True,10484.333333
...,...,...,...,...,...,...,...,...,...,...
16823,fec479d0202d6e1e3f051a9ee902ff5d,33,18089,43300,59,62,23,31-33,True,9908.312500
16825,fec479d0202d6e1e3f051a9ee902ff5d,34,18089,43300,34,23,62,61,True,9908.312500
16827,fec479d0202d6e1e3f051a9ee902ff5d,35,18089,43300,27,62,23,72,True,9908.312500
16829,fec479d0202d6e1e3f051a9ee902ff5d,36,18089,43300,28,23,62,72,True,9908.312500


In [23]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [24]:
temp_df = pd.DataFrame(columns=list(relevant_unemployment_df.columns))
my_ids = list(set(relevant_unemployment_df['uu_id']))
for UUid in my_ids:
    ctyfips = None
    wage_data = None
    tract_vote = [None, 0]
    top_cat_1_vote = [None, 0]
    top_cat_2_vote = [None, 0]
    top_cat_3_vote = [None, 0]
    total_claims_avg = []
    total_claims_num = 0
    
    for week in range(1,PRED_WEEK):
        row = relevant_unemployment_df.loc[((relevant_unemployment_df.uu_id==UUid) 
                                      & (relevant_unemployment_df.week_number == week)),]
        if len(row) == 1:
            temp_df = pd.concat([temp_df, row])
            
            ### Store information about values to infer for rows without data
            ## Consistent within uu_id values
            ctyfips = row['countyfips'].values[0]
            wage_data = row['wage_data'].values[0]
            
            ## Inconsistent within uu_id values
            # Categorical 

            if ((tract_vote[0] == None) or ((tract_vote[0] != row['tract'].values[0]) and (tract_vote[1] == 1))):
                tract_vote = [row['tract'].values[0], 1]
            elif tract_vote[0] == row['tract'].values[0]:
                tract_vote[1] += 1
            else:
                tract_vote[1] -= 1
                
            if ((top_cat_1_vote[0] == None) or ((top_cat_1_vote[0] != row['top_category_employer1'].values[0]) and (top_cat_1_vote[1] == 1))):
                top_cat_1_vote = [row['top_category_employer1'].values[0], 1]
            elif top_cat_1_vote[0] == row['top_category_employer1'].values[0]:
                top_cat_1_vote[1] += 1
            else:
                top_cat_1_vote[1] -= 1
                
            if ((top_cat_2_vote[0] == None) or ((top_cat_2_vote[0] != row['top_category_employer2'].values[0]) and (top_cat_2_vote[1] == 1))):
                top_cat_2_vote = [row['top_category_employer2'].values[0], 1]
            elif top_cat_2_vote[0] == row['top_category_employer2'].values[0]:
                top_cat_2_vote[1] += 1
            else:
                top_cat_2_vote[1] -= 1
                
            if ((top_cat_3_vote[0] == None) or ((top_cat_3_vote[0] != row['top_category_employer3'].values[0]) and (top_cat_3_vote[1] == 1))):
                top_cat_3_vote = [row['top_category_employer3'].values[0], 1]
            elif top_cat_3_vote[0] == row['top_category_employer3'].values[0]:
                top_cat_3_vote[1] += 1
            else:
                top_cat_3_vote[1] -= 1
                
            # Numerical
            total_claims_avg.append(row['total_claims'].values[0])
            if len(total_claims_avg) > 0:
                total_claims_num = np.mean(total_claims_avg)

            continue
        temp_df = pd.concat([temp_df, pd.DataFrame.from_records([{'uu_id': UUid, 
                                                                  'week_number': week, 
                                                                  'countyfips': ctyfips, 
                                                                  'tract': tract_vote[0],
                                                                  'total_claims': total_claims_num, 
                                                                  'top_category_employer1': top_cat_1_vote[0],
                                                                  'top_category_employer2': top_cat_2_vote[0], 
                                                                  'top_category_employer3': top_cat_3_vote[0],
                                                                  'real_data': False, 
                                                                  'wage_data': wage_data}])])

In [25]:
temp_df.reset_index(inplace=True)
temp_df.drop('index', axis=1,inplace=True)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20263 non-null  object 
 3   tract                   20263 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20263 non-null  object 
 6   top_category_employer2  20263 non-null  object 
 7   top_category_employer3  20263 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20263 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [26]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,a5c01f891423feb7623ecfbeea6d103c,1,18097,380700,19.0,23,56,72,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,18097,380700,42.0,56,23,48-49,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,18097,380700,26.0,23,72,56,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,18097,380700,29.0,23,72,56,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,18097,380700,30.0,48-49,23,72,True,11896.5


In [27]:
non_null_cols = ["countyfips", "tract", "top_category_employer1", "top_category_employer2", "top_category_employer3", "wage_data"]

In [28]:
for UUid2 in my_ids[::-1]:
    for week_num in range(PRED_WEEK-1,0,-1):
        row = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)),]
        if row['countyfips'].values[0] == None:
            for col in non_null_cols:
                temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)), col] = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num+1)), col].values[0]

In [29]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  object 
 3   tract                   20475 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  object 
 6   top_category_employer2  20475 non-null  object 
 7   top_category_employer3  20475 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20475 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [30]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,a5c01f891423feb7623ecfbeea6d103c,1,18097,380700,19.0,23,56,72,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,18097,380700,42.0,56,23,48-49,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,18097,380700,26.0,23,72,56,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,18097,380700,29.0,23,72,56,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,18097,380700,30.0,48-49,23,72,True,11896.5


In [31]:
temp_df = temp_df.convert_dtypes()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  string 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  string 
 3   tract                   20475 non-null  string 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  string 
 6   top_category_employer2  20475 non-null  string 
 7   top_category_employer3  20475 non-null  string 
 8   real_data               20475 non-null  boolean
 9   wage_data               20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), string(6)
memory usage: 1.5 MB


In [32]:
bad_wage_ids=set(temp_df.loc[temp_df['wage_data'].isna(),"uu_id"])

In [33]:
data = temp_df
data.rename(columns={"uu_id": "id", "week_number": "week", "countyfips": "fips", "top_category_employer1": "emp1"}, inplace=True)
data.rename(columns={"top_category_employer2": "emp2", "top_category_employer3": "emp3", "real_data": "real", "wage_data":"wages"}, inplace=True)
for col in ['fips', 'tract', 'emp1', 'emp2', 'emp3']:
    data[col] = pd.factorize(data[col])[0]

In [34]:
data.head()

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,a5c01f891423feb7623ecfbeea6d103c,1,0,0,19.0,0,0,0,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,0,0,42.0,1,1,1,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,0,0,26.0,0,2,2,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,0,0,29.0,0,2,2,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,0,0,30.0,2,1,0,True,11896.5


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20475 non-null  string 
 1   week          20475 non-null  Int64  
 2   fips          20475 non-null  int64  
 3   tract         20475 non-null  int64  
 4   total_claims  20475 non-null  Float64
 5   emp1          20475 non-null  int64  
 6   emp2          20475 non-null  int64  
 7   emp3          20475 non-null  int64  
 8   real          20475 non-null  boolean
 9   wages         20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), int64(5), string(1)
memory usage: 1.5 MB


In [36]:
temp_df.head(39)

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,a5c01f891423feb7623ecfbeea6d103c,1,0,0,19.0,0,0,0,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,0,0,42.0,1,1,1,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,0,0,26.0,0,2,2,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,0,0,29.0,0,2,2,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,0,0,30.0,2,1,0,True,11896.5
5,a5c01f891423feb7623ecfbeea6d103c,6,0,0,27.0,0,0,3,True,11896.5
6,a5c01f891423feb7623ecfbeea6d103c,7,0,0,25.0,0,0,0,True,11896.5
7,a5c01f891423feb7623ecfbeea6d103c,8,0,0,37.0,0,2,4,True,11896.5
8,a5c01f891423feb7623ecfbeea6d103c,9,0,0,21.0,0,0,5,True,11896.5
9,a5c01f891423feb7623ecfbeea6d103c,10,0,0,22.0,0,0,1,True,11896.5


In [37]:
abs(data.corr()['total_claims'])

week            0.042895
fips            0.173183
tract           0.036441
total_claims    1.000000
emp1            0.023691
emp2            0.029622
emp3            0.049073
real            0.165613
wages           0.123782
Name: total_claims, dtype: float64

In [38]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [40]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [41]:
results

{'a5c01f891423feb7623ecfbeea6d103c': [[24.185185185185187,
   22.0,
   20.0,
   23.96551724137931,
   15.0,
   22.0,
   13.0,
   23.28125,
   23.28125],
  [24.405979477866584,
   20.940315446541884,
   19.56015456425181,
   24.41335207025233,
   16.74111460056343,
   22.016929413169134,
   23.854452054200852,
   23.331975190918723,
   23.32964567485751]],
 '99c75ff0ed83b04f57a0f99dfc1b05b2': [[20.1,
   20.1,
   20.1,
   21.0,
   20.181818181818183,
   20.181818181818183,
   20.181818181818183,
   20.181818181818183,
   20.181818181818183],
  [20.405691927696513,
   20.38506221391822,
   20.367040878111123,
   10.820009494729865,
   13.494001972090821,
   14.885257138598211,
   15.797350567773464,
   16.441458581672826,
   16.920559577973055]],
 '1f53903c302f96fd0e8a36d02364bbca': [[19.0,
   20.0,
   25.0,
   18.0,
   23.0,
   23.0,
   22.0,
   21.61764705882353,
   21.61764705882353],
  [19.448738333536312,
   21.24956770063727,
   22.39034598417063,
   22.731662380038134,
   22.184386

In [42]:
pd.DataFrame.from_dict(results)

,a5c01f891423feb7623ecfbeea6d103c,99c75ff0ed83b04f57a0f99dfc1b05b2,1f53903c302f96fd0e8a36d02364bbca,2ddb832dc6e28aba32224f559c5f0b63,1ec881fbb0021331b2b29c95049284fb,6b22ecccaddde4516a88c6981373daf3,5ba7caaa7b10591d00eec45bf939284c,93c603c6ec0389076b9f852f9250a231,147837d751cb44500754e9620204f8d6,778946d9696cd6c95f57e8b1e6336389,...,b84d48cb3e210ed3d4ae5522c752bd43,f25446d4d951867ca7b984ad17c6c1e0,cd572640b6e423db8fd7ad50ef0b4473,9883a286690ddb549fd39aff729f83ee,7e2478517261a19d87c0ab2e737fb25f,d6a4947ec7c2a78bd32ba4e30f3bba53,ce8b2c0e11c538921c39c06b298d2393,124fe6d04692c372ffc4f84fc99f480d,99b418342eb86b3aba7061fb1b7584a8,a993d47c7143bed2a187b0cac7d17983
0,"[24.185185185185187, 22.0, 20.0, 23.9655172413...","[20.1, 20.1, 20.1, 21.0, 20.181818181818183, 2...","[19.0, 20.0, 25.0, 18.0, 23.0, 23.0, 22.0, 21....","[11.0, 27.0, 20.0, 13.0, 13.0, 12.0, 18.0, 15....","[37.0, 28.0, 31.0, 22.0, 25.0, 51.0, 30.0, 38....","[12.583333333333334, 12.583333333333334, 12.58...","[20.0, 11.0, 10.0, 10.0, 10.0, 15.0, 12.0, 16....","[29.0, 19.0, 28.0, 15.0, 17.0, 21.0, 15.0, 28....","[25.88235294117647, 25.88235294117647, 25.8823...","[17.0, 12.0, 15.0, 11.0, 12.0, 59.0, 38.0, 18....",...,"[25.0, 11.0, 10.0, 12.0, 12.0, 14.0, 10.0, 16....","[22.0, 16.4, 10.0, 15.333333333333334, 15.3333...","[16.53846153846154, 16.53846153846154, 12.0, 2...","[21.8125, 21.8125, 12.0, 13.0, 20.777777777777...","[21.0, 19.0, 18.0, 19.0, 22.0, 22.0, 21.0, 23....","[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[13.0, 11.0, 13.65, 13.65, 13.65, 11.0, 10.0, ...","[22.0, 18.0, 16.0, 15.0, 15.0, 100.0, 13.0, 24...","[45.0, 28.26086956521739, 28.26086956521739, 2...","[20.0, 29.0, 42.0, 23.0, 29.0, 20.0, 21.0, 29...."
1,"[24.405979477866584, 20.940315446541884, 19.56...","[20.405691927696513, 20.38506221391822, 20.367...","[19.448738333536312, 21.24956770063727, 22.390...","[16.12955383170629, 15.524676218628883, 15.403...","[27.365195643894367, 37.10643763409462, 39.392...","[12.42469581689511, 12.432351342624315, 12.439...","[16.05510659303884, 16.476179033314356, 19.929...","[32.65395884308964, 34.33863001887221, 32.6141...","[27.253357579851823, 27.19410847217793, 27.139...","[17.23326487839222, 18.29741029627621, 18.7268...",...,"[19.48148743617292, 15.210233790834668, 25.400...","[-0.43075626285443747, 16.90863576217732, 17.0...","[15.516224238988569, 15.594302898502402, 14.44...","[19.903329641184097, 20.135951557531172, 22.78...","[25.806446230944204, 25.802500606514513, 23.74...","[15.331006507854909, 15.02211440866813, 13.932...","[13.324319881590782, 13.538688705069944, 13.26...","[21.042728250620712, 20.731856198515743, 20.72...","[58.94681722205132, 22.939291371963918, 23.299...","[32.732894919917726, 29.736283667763928, 28.20..."


In [43]:
pd.DataFrame.from_dict(results, orient='index')

,0,1
a5c01f891423feb7623ecfbeea6d103c,"[24.185185185185187, 22.0, 20.0, 23.9655172413...","[24.405979477866584, 20.940315446541884, 19.56..."
99c75ff0ed83b04f57a0f99dfc1b05b2,"[20.1, 20.1, 20.1, 21.0, 20.181818181818183, 2...","[20.405691927696513, 20.38506221391822, 20.367..."
1f53903c302f96fd0e8a36d02364bbca,"[19.0, 20.0, 25.0, 18.0, 23.0, 23.0, 22.0, 21....","[19.448738333536312, 21.24956770063727, 22.390..."
2ddb832dc6e28aba32224f559c5f0b63,"[11.0, 27.0, 20.0, 13.0, 13.0, 12.0, 18.0, 15....","[16.12955383170629, 15.524676218628883, 15.403..."
1ec881fbb0021331b2b29c95049284fb,"[37.0, 28.0, 31.0, 22.0, 25.0, 51.0, 30.0, 38....","[27.365195643894367, 37.10643763409462, 39.392..."
...,...,...
d6a4947ec7c2a78bd32ba4e30f3bba53,"[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[15.331006507854909, 15.02211440866813, 13.932..."
ce8b2c0e11c538921c39c06b298d2393,"[13.0, 11.0, 13.65, 13.65, 13.65, 11.0, 10.0, ...","[13.324319881590782, 13.538688705069944, 13.26..."
124fe6d04692c372ffc4f84fc99f480d,"[22.0, 18.0, 16.0, 15.0, 15.0, 100.0, 13.0, 24...","[21.042728250620712, 20.731856198515743, 20.72..."
99b418342eb86b3aba7061fb1b7584a8,"[45.0, 28.26086956521739, 28.26086956521739, 2...","[58.94681722205132, 22.939291371963918, 23.299..."


In [44]:
pd.DataFrame.from_dict(results, orient='index').columns = ['actual', 'prediction']

In [45]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df

,actual,prediction
a5c01f891423feb7623ecfbeea6d103c,"[24.185185185185187, 22.0, 20.0, 23.9655172413...","[24.405979477866584, 20.940315446541884, 19.56..."
99c75ff0ed83b04f57a0f99dfc1b05b2,"[20.1, 20.1, 20.1, 21.0, 20.181818181818183, 2...","[20.405691927696513, 20.38506221391822, 20.367..."
1f53903c302f96fd0e8a36d02364bbca,"[19.0, 20.0, 25.0, 18.0, 23.0, 23.0, 22.0, 21....","[19.448738333536312, 21.24956770063727, 22.390..."
2ddb832dc6e28aba32224f559c5f0b63,"[11.0, 27.0, 20.0, 13.0, 13.0, 12.0, 18.0, 15....","[16.12955383170629, 15.524676218628883, 15.403..."
1ec881fbb0021331b2b29c95049284fb,"[37.0, 28.0, 31.0, 22.0, 25.0, 51.0, 30.0, 38....","[27.365195643894367, 37.10643763409462, 39.392..."
...,...,...
d6a4947ec7c2a78bd32ba4e30f3bba53,"[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[15.331006507854909, 15.02211440866813, 13.932..."
ce8b2c0e11c538921c39c06b298d2393,"[13.0, 11.0, 13.65, 13.65, 13.65, 11.0, 10.0, ...","[13.324319881590782, 13.538688705069944, 13.26..."
124fe6d04692c372ffc4f84fc99f480d,"[22.0, 18.0, 16.0, 15.0, 15.0, 100.0, 13.0, 24...","[21.042728250620712, 20.731856198515743, 20.72..."
99b418342eb86b3aba7061fb1b7584a8,"[45.0, 28.26086956521739, 28.26086956521739, 2...","[58.94681722205132, 22.939291371963918, 23.299..."


In [46]:
print("Hello World :)")

Hello World :)


In [47]:
get_ipython().system('pip install db-dtypes')

In [48]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import matplotlib.pyplot as plt
import numpy as np

In [49]:
# Constants
PRED_WEEK = 40

In [50]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [51]:
query_get_tables = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""
query_job = bigquery_client.query(query_get_tables)
query_job.to_dataframe().head()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,ironhacks-data,ironhacks_competition,wage_data,BASE TABLE,YES,NO,2022-11-29 17:00:10.578000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
1,ironhacks-data,ironhacks_competition,prediction_list,BASE TABLE,YES,NO,2022-11-29 17:00:10.722000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
2,ironhacks-data,ironhacks_competition,unemployment_data,BASE TABLE,YES,NO,2022-11-29 17:00:14.588000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT


In [52]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [53]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0


In [54]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [55]:
query_job = bigquery_client.query(query)
prediction_list_data = query_job.to_dataframe()
prediction_list_data.head(2)

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44


In [56]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [57]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head(2)

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.00
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.25


In [58]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
WHERE uu_id IN (SELECT uu_id
                FROM `ironhacks-data.ironhacks_competition.prediction_list`
                WHERE week_number = (SELECT MAX(week_number) FROM `ironhacks-data.ironhacks_competition.prediction_list`))
ORDER BY uu_id,week_number
"""

In [59]:
query_job = bigquery_client.query(query)
relevant_unemployment_df = query_job.to_dataframe()
relevant_unemployment_df.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [60]:
relevant_unemployment_df.head(3)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10
2,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [61]:
# Drop duplicate columns
relevant_unemployment_df.drop_duplicates(inplace=True)
relevant_unemployment_df.drop(['timeperiod', 'tract_name'], axis=1, inplace=True)

In [62]:
# Drop columns with excessive null values
# NOTE: Revisit this, these columns may still be useful, especially those that aren't missing too many values
relevant_unemployment_df.dropna(axis='columns', inplace=True)
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()
relevant_unemployment_df['countyfips'] = relevant_unemployment_df['countyfips'].astype(str)
relevant_unemployment_df['tract'] = relevant_unemployment_df['tract'].astype(str)
relevant_unemployment_df['real_data'] = True
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()

Add rows such that every uu_id has rows for weeks 1:max(week_number)

In [63]:
# Add Wage data
relevant_unemployment_df['wage_data'] = relevant_unemployment_df['uu_id'].map(wage_data.set_index('uu_id')['average_wage'])
relevant_unemployment_df.info()
bad_wage_uu_ids = list(set(relevant_unemployment_df.loc[relevant_unemployment_df['wage_data'].isna(),'uu_id']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13977 entries, 0 to 16831
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   13977 non-null  string 
 1   week_number             13977 non-null  Int64  
 2   countyfips              13977 non-null  string 
 3   tract                   13977 non-null  string 
 4   total_claims            13977 non-null  Int64  
 5   top_category_employer1  13977 non-null  string 
 6   top_category_employer2  13977 non-null  string 
 7   top_category_employer3  13977 non-null  string 
 8   real_data               13977 non-null  boolean
 9   wage_data               13977 non-null  float64
dtypes: Int64(2), boolean(1), float64(1), string(6)
memory usage: 1.1 MB


In [64]:
for uuid in set(relevant_unemployment_df["uu_id"]):
    num_tracts = 0
    tracts=[]
    for tract in set(relevant_unemployment_df.loc[relevant_unemployment_df["uu_id"]==uuid,'wage_data']):
        num_tracts += 1
        tracts.append(tract)
    if num_tracts > 1:
        print(uuid,num_tracts,tracts)
relevant_unemployment_df

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,001cd9ae23064d7f0fd3cd327c873d8d,31,18099,20102,34,31-33,23,81,True,10484.333333
1,001cd9ae23064d7f0fd3cd327c873d8d,33,18099,20102,10,31-33,23,81,True,10484.333333
3,001cd9ae23064d7f0fd3cd327c873d8d,34,18099,20102,25,62,31-33,23,True,10484.333333
5,001cd9ae23064d7f0fd3cd327c873d8d,35,18099,20102,10,31-33,61,23,True,10484.333333
7,001cd9ae23064d7f0fd3cd327c873d8d,36,18099,20102,10,31-33,23,81,True,10484.333333
...,...,...,...,...,...,...,...,...,...,...
16823,fec479d0202d6e1e3f051a9ee902ff5d,33,18089,43300,59,62,23,31-33,True,9908.312500
16825,fec479d0202d6e1e3f051a9ee902ff5d,34,18089,43300,34,23,62,61,True,9908.312500
16827,fec479d0202d6e1e3f051a9ee902ff5d,35,18089,43300,27,62,23,72,True,9908.312500
16829,fec479d0202d6e1e3f051a9ee902ff5d,36,18089,43300,28,23,62,72,True,9908.312500


In [65]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [66]:
temp_df = pd.DataFrame(columns=list(relevant_unemployment_df.columns))
my_ids = list(set(relevant_unemployment_df['uu_id']))
for UUid in my_ids:
    ctyfips = None
    wage_data = None
    tract_vote = [None, 0]
    top_cat_1_vote = [None, 0]
    top_cat_2_vote = [None, 0]
    top_cat_3_vote = [None, 0]
    total_claims_avg = []
    total_claims_num = 0
    
    for week in range(1,PRED_WEEK):
        row = relevant_unemployment_df.loc[((relevant_unemployment_df.uu_id==UUid) 
                                      & (relevant_unemployment_df.week_number == week)),]
        if len(row) == 1:
            temp_df = pd.concat([temp_df, row])
            
            ### Store information about values to infer for rows without data
            ## Consistent within uu_id values
            ctyfips = row['countyfips'].values[0]
            wage_data = row['wage_data'].values[0]
            
            ## Inconsistent within uu_id values
            # Categorical 

            if ((tract_vote[0] == None) or ((tract_vote[0] != row['tract'].values[0]) and (tract_vote[1] == 1))):
                tract_vote = [row['tract'].values[0], 1]
            elif tract_vote[0] == row['tract'].values[0]:
                tract_vote[1] += 1
            else:
                tract_vote[1] -= 1
                
            if ((top_cat_1_vote[0] == None) or ((top_cat_1_vote[0] != row['top_category_employer1'].values[0]) and (top_cat_1_vote[1] == 1))):
                top_cat_1_vote = [row['top_category_employer1'].values[0], 1]
            elif top_cat_1_vote[0] == row['top_category_employer1'].values[0]:
                top_cat_1_vote[1] += 1
            else:
                top_cat_1_vote[1] -= 1
                
            if ((top_cat_2_vote[0] == None) or ((top_cat_2_vote[0] != row['top_category_employer2'].values[0]) and (top_cat_2_vote[1] == 1))):
                top_cat_2_vote = [row['top_category_employer2'].values[0], 1]
            elif top_cat_2_vote[0] == row['top_category_employer2'].values[0]:
                top_cat_2_vote[1] += 1
            else:
                top_cat_2_vote[1] -= 1
                
            if ((top_cat_3_vote[0] == None) or ((top_cat_3_vote[0] != row['top_category_employer3'].values[0]) and (top_cat_3_vote[1] == 1))):
                top_cat_3_vote = [row['top_category_employer3'].values[0], 1]
            elif top_cat_3_vote[0] == row['top_category_employer3'].values[0]:
                top_cat_3_vote[1] += 1
            else:
                top_cat_3_vote[1] -= 1
                
            # Numerical
            total_claims_avg.append(row['total_claims'].values[0])
            if len(total_claims_avg) > 0:
                total_claims_num = np.mean(total_claims_avg)

            continue
        temp_df = pd.concat([temp_df, pd.DataFrame.from_records([{'uu_id': UUid, 
                                                                  'week_number': week, 
                                                                  'countyfips': ctyfips, 
                                                                  'tract': tract_vote[0],
                                                                  'total_claims': total_claims_num, 
                                                                  'top_category_employer1': top_cat_1_vote[0],
                                                                  'top_category_employer2': top_cat_2_vote[0], 
                                                                  'top_category_employer3': top_cat_3_vote[0],
                                                                  'real_data': False, 
                                                                  'wage_data': wage_data}])])

In [67]:
temp_df.reset_index(inplace=True)
temp_df.drop('index', axis=1,inplace=True)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20263 non-null  object 
 3   tract                   20263 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20263 non-null  object 
 6   top_category_employer2  20263 non-null  object 
 7   top_category_employer3  20263 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20263 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [68]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,a5c01f891423feb7623ecfbeea6d103c,1,18097,380700,19.0,23,56,72,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,18097,380700,42.0,56,23,48-49,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,18097,380700,26.0,23,72,56,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,18097,380700,29.0,23,72,56,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,18097,380700,30.0,48-49,23,72,True,11896.5


In [69]:
non_null_cols = ["countyfips", "tract", "top_category_employer1", "top_category_employer2", "top_category_employer3", "wage_data"]

In [70]:
for UUid2 in my_ids[::-1]:
    for week_num in range(PRED_WEEK-1,0,-1):
        row = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)),]
        if row['countyfips'].values[0] == None:
            for col in non_null_cols:
                temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)), col] = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num+1)), col].values[0]

In [71]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  object 
 3   tract                   20475 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  object 
 6   top_category_employer2  20475 non-null  object 
 7   top_category_employer3  20475 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20475 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [72]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,a5c01f891423feb7623ecfbeea6d103c,1,18097,380700,19.0,23,56,72,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,18097,380700,42.0,56,23,48-49,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,18097,380700,26.0,23,72,56,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,18097,380700,29.0,23,72,56,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,18097,380700,30.0,48-49,23,72,True,11896.5


In [73]:
temp_df = temp_df.convert_dtypes()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  string 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  string 
 3   tract                   20475 non-null  string 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  string 
 6   top_category_employer2  20475 non-null  string 
 7   top_category_employer3  20475 non-null  string 
 8   real_data               20475 non-null  boolean
 9   wage_data               20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), string(6)
memory usage: 1.5 MB


In [74]:
bad_wage_ids=set(temp_df.loc[temp_df['wage_data'].isna(),"uu_id"])

In [75]:
data = temp_df
data.rename(columns={"uu_id": "id", "week_number": "week", "countyfips": "fips", "top_category_employer1": "emp1"}, inplace=True)
data.rename(columns={"top_category_employer2": "emp2", "top_category_employer3": "emp3", "real_data": "real", "wage_data":"wages"}, inplace=True)
for col in ['fips', 'tract', 'emp1', 'emp2', 'emp3']:
    data[col] = pd.factorize(data[col])[0]

In [76]:
data.head()

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,a5c01f891423feb7623ecfbeea6d103c,1,0,0,19.0,0,0,0,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,0,0,42.0,1,1,1,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,0,0,26.0,0,2,2,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,0,0,29.0,0,2,2,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,0,0,30.0,2,1,0,True,11896.5


In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20475 non-null  string 
 1   week          20475 non-null  Int64  
 2   fips          20475 non-null  int64  
 3   tract         20475 non-null  int64  
 4   total_claims  20475 non-null  Float64
 5   emp1          20475 non-null  int64  
 6   emp2          20475 non-null  int64  
 7   emp3          20475 non-null  int64  
 8   real          20475 non-null  boolean
 9   wages         20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), int64(5), string(1)
memory usage: 1.5 MB


In [78]:
temp_df.head(39)

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,a5c01f891423feb7623ecfbeea6d103c,1,0,0,19.0,0,0,0,True,11896.5
1,a5c01f891423feb7623ecfbeea6d103c,2,0,0,42.0,1,1,1,True,11896.5
2,a5c01f891423feb7623ecfbeea6d103c,3,0,0,26.0,0,2,2,True,11896.5
3,a5c01f891423feb7623ecfbeea6d103c,4,0,0,29.0,0,2,2,False,11896.5
4,a5c01f891423feb7623ecfbeea6d103c,5,0,0,30.0,2,1,0,True,11896.5
5,a5c01f891423feb7623ecfbeea6d103c,6,0,0,27.0,0,0,3,True,11896.5
6,a5c01f891423feb7623ecfbeea6d103c,7,0,0,25.0,0,0,0,True,11896.5
7,a5c01f891423feb7623ecfbeea6d103c,8,0,0,37.0,0,2,4,True,11896.5
8,a5c01f891423feb7623ecfbeea6d103c,9,0,0,21.0,0,0,5,True,11896.5
9,a5c01f891423feb7623ecfbeea6d103c,10,0,0,22.0,0,0,1,True,11896.5


In [79]:
abs(data.corr()['total_claims'])

week            0.042895
fips            0.173183
tract           0.036441
total_claims    1.000000
emp1            0.023691
emp2            0.029622
emp3            0.049073
real            0.165613
wages           0.123782
Name: total_claims, dtype: float64

In [80]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]

In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [82]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [83]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df

,actual,prediction
a5c01f891423feb7623ecfbeea6d103c,"[24.185185185185187, 22.0, 20.0, 23.9655172413...","[24.405979477866584, 20.940315446541884, 19.56..."
99c75ff0ed83b04f57a0f99dfc1b05b2,"[20.1, 20.1, 20.1, 21.0, 20.181818181818183, 2...","[20.405691927696513, 20.38506221391822, 20.367..."
1f53903c302f96fd0e8a36d02364bbca,"[19.0, 20.0, 25.0, 18.0, 23.0, 23.0, 22.0, 21....","[19.448738333536312, 21.24956770063727, 22.390..."
2ddb832dc6e28aba32224f559c5f0b63,"[11.0, 27.0, 20.0, 13.0, 13.0, 12.0, 18.0, 15....","[16.12955383170629, 15.524676218628883, 15.403..."
1ec881fbb0021331b2b29c95049284fb,"[37.0, 28.0, 31.0, 22.0, 25.0, 51.0, 30.0, 38....","[27.365195643894367, 37.10643763409462, 39.392..."
...,...,...
d6a4947ec7c2a78bd32ba4e30f3bba53,"[14.0, 14.0, 23.0, 15.5, 15.5, 11.0, 38.0, 17....","[15.331006507854909, 15.02211440866813, 13.932..."
ce8b2c0e11c538921c39c06b298d2393,"[13.0, 11.0, 13.65, 13.65, 13.65, 11.0, 10.0, ...","[13.324319881590782, 13.538688705069944, 13.26..."
124fe6d04692c372ffc4f84fc99f480d,"[22.0, 18.0, 16.0, 15.0, 15.0, 100.0, 13.0, 24...","[21.042728250620712, 20.731856198515743, 20.72..."
99b418342eb86b3aba7061fb1b7584a8,"[45.0, 28.26086956521739, 28.26086956521739, 2...","[58.94681722205132, 22.939291371963918, 23.299..."


In [84]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df.explode(['actual','prediction])

SyntaxError: EOL while scanning string literal (2748506027.py, line 3)

In [85]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df.explode(['actual','prediction'])

,actual,prediction
a5c01f891423feb7623ecfbeea6d103c,24.185185,24.405979
a5c01f891423feb7623ecfbeea6d103c,22.0,20.940315
a5c01f891423feb7623ecfbeea6d103c,20.0,19.560155
a5c01f891423feb7623ecfbeea6d103c,23.965517,24.413352
a5c01f891423feb7623ecfbeea6d103c,15.0,16.741115
...,...,...
a993d47c7143bed2a187b0cac7d17983,29.0,29.368245
a993d47c7143bed2a187b0cac7d17983,20.0,32.840147
a993d47c7143bed2a187b0cac7d17983,21.0,34.117499
a993d47c7143bed2a187b0cac7d17983,29.114286,31.278692


In [86]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
weeks

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39]

In [87]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
PRED_WEEK

40

In [88]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
PRED_WEEK-10

30

In [89]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]
test_weeks

[31, 32, 33, 34, 35, 36, 37, 38, 39]

In [90]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-11:]
test_weeks

[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]

In [91]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [92]:
results = dict()
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
        results[pred_id] = [actual[pred_id], predictions[pred_id]]
    
    # Error metrics
    # MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    # MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [93]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df=results_df.explode(['actual','prediction'])
results_df

,actual,prediction
a5c01f891423feb7623ecfbeea6d103c,15.0,26.964983
a5c01f891423feb7623ecfbeea6d103c,24.185185,24.405979
a5c01f891423feb7623ecfbeea6d103c,22.0,20.940315
a5c01f891423feb7623ecfbeea6d103c,20.0,19.560155
a5c01f891423feb7623ecfbeea6d103c,23.965517,24.413352
...,...,...
a993d47c7143bed2a187b0cac7d17983,29.0,29.368245
a993d47c7143bed2a187b0cac7d17983,20.0,32.840147
a993d47c7143bed2a187b0cac7d17983,21.0,34.117499
a993d47c7143bed2a187b0cac7d17983,29.114286,31.278692


In [94]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df=results_df.explode(['actual','prediction'])
#results_df['week'] = []
1:10

SyntaxError: illegal target for annotation (3944715655.py, line 5)

In [95]:
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = ['actual', 'prediction']
results_df=results_df.explode(['actual','prediction'])
#results_df['week'] = []
[1:10]

SyntaxError: invalid syntax (2934792075.py, line 5)